# Job listings pipeline

The idea is to read many job descriptions and get some specifics plus some vibes about the company:
- Benefits
- How the company talks about themselves
- Technology/expectations
- Leveling expectations

In [1]:
from core import Seed, init

init()

In [2]:
from core import make_experiment_dir

target = Seed.init("Rippling")
experiment_dir = make_experiment_dir(target)

In [7]:
from google_search import search

list(search(f'site:www.indeed.com link:www.rippling.com', num=10))

2024-08-29 11:03:44.930 | DEBUG    | google_search:search:58 - Google search results: {'kind': 'customsearch#search', 'url': {'type': 'application/json', 'template': 'https://www.googleapis.com/customsearch/v1?q={searchTerms}&num={count?}&start={startIndex?}&lr={language?}&safe={safe?}&cx={cx?}&sort={sort?}&filter={filter?}&gl={gl?}&cr={cr?}&googlehost={googleHost?}&c2coff={disableCnTwTranslation?}&hq={hq?}&hl={hl?}&siteSearch={siteSearch?}&siteSearchFilter={siteSearchFilter?}&exactTerms={exactTerms?}&excludeTerms={excludeTerms?}&linkSite={linkSite?}&orTerms={orTerms?}&dateRestrict={dateRestrict?}&lowRange={lowRange?}&highRange={highRange?}&searchType={searchType}&fileType={fileType?}&rights={rights?}&imgSize={imgSize?}&imgType={imgType?}&imgColorType={imgColorType?}&imgDominantColor={imgDominantColor?}&alt=json'}, 'queries': {'request': [{'title': 'Google Custom Search - site:www.indeed.com link:www.rippling.com', 'totalResults': '84', 'searchTerms': 'site:www.indeed.com link:www.ripp

[SearchResult(title='Applicant Tracking System | ATS Integrations', link='https://www.indeed.com/hire/ats-integration', snippet='ATS Sync integrations seamlessly transfer job and candidate data between Indeed and your ATS. We integrate with over 300 ATS platforms globally.', formattedUrl='https://www.indeed.com/hire/ats-integration'),
 SearchResult(title='25+ ATS Jobs, Employment in Raleigh, NC August 21, 2024| Indeed ...', link='https://www.indeed.com/q-ats-l-raleigh%2C-nc-jobs.html', snippet="Please use this link to apply: https://ats.rippling.com/lexitas-open-positions/jobs/f5e13aff-d493-44bf-a6d5-78c67a24e493. We're looking for purpose-driven\xa0...", formattedUrl='https://www.indeed.com/q-ats-l-raleigh%2C-nc-jobs.html'),
 SearchResult(title='Preppy Pet West Houston Careers and Employment | Indeed.com', link='https://www.indeed.com/cmp/Preppy-Pet-West-Houston', snippet='About the company · Industry. Pet Care & Veterinary · Headquarters. 16150 Rippling Water Dr. ... Show more · Link

In [3]:
import indeed

summary = await indeed.run(target)

2024-08-29 10:57:56.152 | DEBUG    | google_search:search:58 - Google search results: {'kind': 'customsearch#search', 'url': {'type': 'application/json', 'template': 'https://www.googleapis.com/customsearch/v1?q={searchTerms}&num={count?}&start={startIndex?}&lr={language?}&safe={safe?}&cx={cx?}&sort={sort?}&filter={filter?}&gl={gl?}&cr={cr?}&googlehost={googleHost?}&c2coff={disableCnTwTranslation?}&hq={hq?}&hl={hl?}&siteSearch={siteSearch?}&siteSearchFilter={siteSearchFilter?}&exactTerms={exactTerms?}&excludeTerms={excludeTerms?}&linkSite={linkSite?}&orTerms={orTerms?}&dateRestrict={dateRestrict?}&lowRange={lowRange?}&highRange={highRange?}&searchType={searchType}&fileType={fileType?}&rights={rights?}&imgSize={imgSize?}&imgType={imgType?}&imgColorType={imgColorType?}&imgDominantColor={imgDominantColor?}&alt=json'}, 'queries': {'request': [{'title': 'Google Custom Search - site:www.indeed.com/cmp "Rippling"', 'totalResults': '7', 'searchTerms': 'site:www.indeed.com/cmp "Rippling"', 'cou

AssertionError: No Indeed jobs found for Rippling

In [ ]:
summary

AIMessage(content='# Summary of Key Information for Staff Machine Learning Engineer at Pomelo Care\n\n## Company Overview\nPomelo Care is a mission-driven startup focused on improving healthcare outcomes for mothers and babies through a technology-driven care platform. The company emphasizes evidence-based practices and aims to reduce preterm births, NICU admissions, c-sections, and maternal mortality. Their approach includes early patient engagement, individualized risk assessments, and coordinated virtual care throughout pregnancy and postpartum.\n\n## Unique Aspects of Roles at Pomelo Care\n- **Mission-Driven Environment**: Employees are part of a team that is dedicated to making a significant impact on maternal and child health.\n- **Multi-Disciplinary Collaboration**: The company fosters a collaborative environment where clinicians, engineers, and problem solvers work together.\n- **Focus on Data-Driven Decision Making**: The company values data-driven approaches in all aspects of

In [ ]:
with open(f"{experiment_dir}/indeed.md", "w") as f:
    f.write(summary.content)